In [87]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

In [6]:
df.head(1)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0


# Data Exploration

In [39]:
df.yr_renovated.value_counts()

0       19171
2014       86
2005       34
2003       34
2013       33
        ...  
1934        1
1951        1
1948        1
1954        1
1956        1
Name: yr_renovated, Length: 70, dtype: int64

In [41]:
df.bedrooms.value_counts()

3     9101
4     6354
2     2567
5     1478
6      251
1      179
7       36
8       12
0       11
9        6
10       3
11       1
33       1
Name: bedrooms, dtype: int64

# Pre-processing

In [88]:
df = (df

    .drop(labels = 'id', axis = 1)

    .assign(date = lambda x: x.date.str.slice(start = 0, stop = 8)

     , year = lambda x: x.date.str.slice(start=0, stop=4)

     , month = lambda x: x.date.str.slice(start=4, stop = 6)

     , day = lambda x: x.date.str.slice(start=6, stop= 8))

    .drop(labels = 'date', axis = 1)

)

In [89]:
df['month'] = df['month'].astype(int)

In [90]:
df['day'] = df['day'].astype(int)

In [91]:
df['year'] = df['year'].astype(int)

In [92]:
from sklearn.preprocessing import MinMaxScaler

In [93]:
df = df.assign(lat_normalized = MinMaxScaler().fit_transform(np.array(df.lat).reshape(-1,1)))


# Model

In [47]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [94]:
target = df.price
features = df.drop(columns="price",axis=1)

In [95]:
X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=.20,train_size=.80)

In [96]:
model = XGBRegressor()
model.fit(X_train, y_train)

[19:36:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [97]:
predictions = model.predict(X_test)
predictions

array([377056.56, 311579.  , 837127.6 , ..., 532821.8 , 513348.6 ,
       706746.75], dtype=float32)

In [98]:
# Compute the Root Mean Squared Error of the predictions
from sklearn.metrics import mean_squared_error

result = mean_squared_error(y_test, predictions, squared=False)
result


127477.63627218484

In [99]:
feature_df = pd.DataFrame(

    {'features': features.columns, 'importance': model.feature_importances_})

feature_df.sort_values('importance', ascending=False).head(6)

,features,importance
8,grade,0.357916
2,sqft_living,0.191289
5,waterfront,0.078301
6,view,0.074615
14,lat,0.061170
15,long,0.046861
